# Prerequisites

- Python 3.10.4

> Warning: Installation from conda environment may take few minutes

Configuring conda environment
```cmd
conda create -n ca2_env
conda activate ca2_env
```

Installing jupyter notebook on ca2_env environment
```cmd
conda install jupyter notebook
python -m ipykernel install --name ca2_env
```

Run jupiter 
```cmd
jupyter notebook
```

How crop and cereals contributes to the agricultural value added indices in Ireland and Europe.

In [100]:
import pandas as pd
import numpy as np

crop_datasets = {
    "fao-crop-residues"                     : "../data/fao/EU-Crop-Residues.csv",
    "fao-crop-production-indices"           : "../data/fao/EU-Crops-Production-Indices.csv",
    "fao-employment-indicators-rural"       : "../data/fao/EU-Employment-Indicators-Rural.csv",
    "fao-cereals-producer-prices"           : "../data/fao/EU-Producer-Prices-Cereals.csv",
    "fao-land-use"                          : "../data/fao/EU-Land-Use.csv",
    "fao-total-energy-use"                  : "../data/fao/EU-Total-Energy-Use.csv",
    "fao-cereals-export-import"             : "../data/fao/EU-Trade-Indices-Cereals-Export-Import.csv",
    "fao-agriculture-value-added"           : "../data/fao/EU-Value-Added-Agriculture.csv",
    "fadn-subsides"                         : "../data/fadn/subsides-year-ms-farmingtype.xlsx",
    "ilostat-employment-working-hours-rural": "../data/ilostat/HOW_TEMP_SEX_OCU_GEO_NB_A-filtered-2022-05-12.xlsx",
    "fadn-summary"                          : "../data/fadn/sumary-year-ms-region-so.xlsx",
}


In [177]:
def init_df():
    eu = ["BE","BG","CY","CZ","DK","DE","EL","ES","EE","FR","HR","HU","IE","IT","LT","LV","LU","MT","NL","AT","PL","PT","RO","FI","SE","SK","SI"]
    years = np.arange(start=2000,stop=2022,step=1)
    # https://www.adamsmith.haus/python/answers/how-to-get-all-element-combinations-of-two-numpy-arrays-in-python
    data = np.array(np.meshgrid(eu, years)).T.reshape(-1, 2)
    df = pd.DataFrame(data, columns=["country","year"])
    df["year"] =df.year.astype(int)
    return df

agriculture_df = init_df()

In [178]:
def get_fao_dataset(dataset_name, element_code, value_col_name, aggfunc="mean", usecols=["Area Code (ISO2)","Year","Element Code","Item","Value"], query = None):
    if(query == None):
        query = f"`Element Code` == {element_code}"

    df = pd.read_csv(crop_datasets[dataset_name], usecols=usecols);
    agg_df = df.query(query).groupby(["Area Code (ISO2)","Year"])["Value"].agg(aggfunc).reset_index()
    agg_df.columns = ["country","year",value_col_name]
    return agg_df
agriculture_df = agriculture_df.merge(get_fao_dataset("fao-crop-residues", 72392, "crop-residues"), how="left")
agriculture_df = agriculture_df.merge(get_fao_dataset("fao-crop-production-indices", 432, "production_indices"), how="left")
agriculture_df = agriculture_df.merge(get_fao_dataset(dataset_name="fao-employment-indicators-rural",
                                                    element_code=21098,
                                                    value_col_name="employment_total_rural_areas",
                                                    usecols=["Area Code (ISO2)","Year","Indicator Code","Sex", "Value"],
                                                    query="`Indicator Code` == 21098 & Sex == 'Total'"), how="left")
agriculture_df = agriculture_df.merge(get_fao_dataset("fao-cereals-producer-prices", 5532, "cereals_produce_price_usd_tonne"), how="left")



In [179]:
agriculture_df

,country,year,crop-residues,production_indices,employment_total_rural_areas,cereals_produce_price_usd_tonne
0,BE,2000,4.602066e+06,106.958571,159.517,99.750000
1,BE,2001,4.266372e+06,89.114286,194.542,95.500000
2,BE,2002,4.727120e+06,97.641429,182.523,86.250000
3,BE,2003,4.524311e+06,100.244286,209.472,120.400000
4,BE,2004,5.241938e+06,97.755714,234.114,113.000000
...,...,...,...,...,...,...
589,SI,2017,6.062981e+05,89.989000,447.229,155.200000
590,SI,2018,7.054626e+05,124.761429,448.700,259.571429
591,SI,2019,7.586285e+05,122.682857,443.512,198.166667
592,SI,2020,NaN,NaN,442.685,244.625000
